### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

outdir = setup("20220711_scenario_projections.ipynb")

E:\Documents\academics\csu\Extracurricular\CO_COVID_Modeling_Team\repo


2022/07/12 09:29:33 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [ ]:
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}
fit_args = {
}
model = do_single_fit(**fit_args, **model_args)

### Create and run scenarios from Feb 2022 to present

In [ ]:
model_args = {
    'base_spec_id': 3014
}
fit_args = {
    'fit_start_date': '2022-03-01',
}

# compute vacc effectiveness
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.3, 0.5, 0.8]:
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for (weak_escape, strong_escape) in [(0.75, 0.1), (0.8, 0.2)]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac)/2, "2022-03-15": (0.34 + 0.66*hosp_frac)/2, "2022-03-30": hosp_frac}
            school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "vals": {},"desc": ""}]
            lt5_vacc_adjust = [{"param": "immunity", "attrs": {'age': '0-19'}, "mults": {}]
            scenario_args_list.append({'params_defs': base_model_args['params_defs'] + school_contact + lt5_vacc_adjust,
                                       'hosp_reporting_frac': hosp_frac,
                                       'tags': {}})


# run the scenarios
models = do_fit_scenarios(base_model_args, scenario_args_list, fit_args, multiprocess=multiprocess)

### Create and run scenarios from Feb 2022 to present

In [ ]:
model_args = {
    #'base_spec_id': 3014,
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json'),
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'regions': ['co'],
    'mobility_mode': None,
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
    'max_step_size': 1.0,
    'ode_method': 'RK45'
}
fit_args = {
    'fit_start_date': '2022-03-01'
}
# compute vacc effectiveness
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.3, 0.5, 0.8]:
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for (weak_escape, strong_escape) in [(0.75, 0.1), (0.8, 0.2)]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac)/2, "2022-03-15": (0.34 + 0.66*hosp_frac)/2, "2022-03-30": hosp_frac}
            school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "vals": {,"desc": ""}]
            lt5_vacc_adjsut = [{"param": "immunity", "attrs": {'age': '0-19'}, "mults": {"2020-01-01": , "2021-12-23": 2, "2022-03-05": 300, "2022-03-9": 400, "2022-03-20": 0},"desc": ""}]
            scenario_args_list.append({'params_defs': base_model_args['params_defs'] + school_contact + ,
                                       'hosp_reporting_frac': hosp_frac,
                                       'tags': {}})


# run the scenarios
models = do_fit_scenarios(base_model_args, scenario_args_list, fit_args, multiprocess=multiprocess)

### Run the report for each fit model

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')